In [1]:
from thesis_initialise import *

In [2]:
from everest.funcy import generic as _generic
class MyArray(_generic.FuncyIncisable):
    def __init__(self, arr, /):
        self.arr = arr
    @property
    def shape(self):
        return self.arr.shape
    def _getitem_broad(self, ind):
        print('broad', ind)
        return self.__class__(self.arr[ind])
    def _getitem_strict(self, ind):
        print('strict', ind)
        return self.__class__(self.arr[ind])
    def _getitem_sub(self, arg0, *argn):
        print('sub', arg0, argn)
        ind = tuple((slice(None), arg0, *argn))
        return self.__class__(self.arr[ind])
    def __repr__(self):
        return f"{self.__class__.__name__}({str(self.arr)})"

In [3]:
myarr = np.random.rand(3, 4, 5, 6, 7)
myinst = MyArray(myarr)

In [4]:
sl = slice(1, 2, 3)
tests = [
    0,
    (0, 0),
    slice(None),
    sl,
    (sl, 0),
    (0, sl),
    (sl, 0, 0),
    (0, 0, sl),
    (0, sl, 0),
    (0, sl, 0, 0),
    (0, 0, sl, 0),
    (0, 0, sl, 0, 0),
    (0, sl, 0, sl, 0),
    (sl, sl, 0, 0, sl),
    ...,
    (..., 0),
    (0, ...),
    (..., 0, 0),
    (0, 0, ...),
    (sl, 0, ...),
    (0, sl, ...),
    (..., sl, 0),
    (..., 0, sl),
    (..., 0, sl, 0),
    (..., sl, sl, sl),
    ]
for test in tests:
    print(test)
    print(' ', np.allclose(myinst[test].arr, myarr[test]))
    print('\n')

0
strict 0
  True


(0, 0)
strict 0
strict 0
  True


slice(None, None, None)
broad slice(None, None, None)
  True


slice(1, 2, 3)
broad slice(1, 2, 3)
  True


(slice(1, 2, 3), 0)
broad slice(1, 2, 3)
sub 0 ()
  True


(0, slice(1, 2, 3))
strict 0
broad slice(1, 2, 3)
  True


(slice(1, 2, 3), 0, 0)
broad slice(1, 2, 3)
sub 0 (0,)
  True


(0, 0, slice(1, 2, 3))
strict 0
strict 0
broad slice(1, 2, 3)
  True


(0, slice(1, 2, 3), 0)
strict 0
broad slice(1, 2, 3)
sub 0 ()
  True


(0, slice(1, 2, 3), 0, 0)
strict 0
broad slice(1, 2, 3)
sub 0 (0,)
  True


(0, 0, slice(1, 2, 3), 0)
strict 0
strict 0
broad slice(1, 2, 3)
sub 0 ()
  True


(0, 0, slice(1, 2, 3), 0, 0)
strict 0
strict 0
broad slice(1, 2, 3)
sub 0 (0,)
  True


(0, slice(1, 2, 3), 0, slice(1, 2, 3), 0)
strict 0
broad slice(1, 2, 3)
sub 0 (slice(1, 2, 3), 0)
  True


(slice(1, 2, 3), slice(1, 2, 3), 0, 0, slice(1, 2, 3))
broad slice(1, 2, 3)
sub slice(1, 2, 3) (0, 0, slice(1, 2, 3))
  True


Ellipsis
  True


(Ellipsis, 0)
b

In [ ]:
np.allclose(myinst[0].arr, myarr[0])

In [ ]:
np.allclose(myinst[:].arr, myarr[:])

In [ ]:
np.allclose(myinst[0, 0].arr, myarr[0, 0])

In [ ]:
np.allclose(myinst[:, 0].arr, myarr[:, 0])

In [ ]:
np.allclose(myinst[0, :, 0].arr, myarr[0, :, 0])

In [ ]:
myinst[..., 0].shape

In [ ]:
myarr[..., 0].shape

In [ ]:
np.allclose(myinst[..., 0].arr, myarr[..., 0])

In [ ]:
np.allclose(myinst[..., 0, 0].arr, myarr[..., 0, 0])

In [ ]:
np.allclose(myinst[0, ...].arr, myarr[0, ...])

In [ ]:
np.allclose(myinst[0, 0, ...].arr, myarr[0, 0, ...])

In [ ]:
np.allclose(myinst[0, ..., 0].arr, myarr[0, ..., 0])

In [ ]:
np.allclose(myinst[0, 0, ..., 0].arr, myarr[0, 0, ..., 0])

In [ ]:
np.allclose(myinst[0, 0, ..., 0].arr, myarr[0, 0, ..., 0])

In [ ]:
myinst[:, 0]

In [ ]:
myarr[:, 0]

In [ ]:
myarr[0, 0]

In [ ]:
myarr[0][0]

In [ ]:
from everest.funcy.utilities import delim_split
tests = [
    (0,),
    (0, 1),
    ((0, 1), 2),
    (0, ...),
    (..., 0),
    (0, 1, ...),
    (..., 0, 1),
    ((0, 1), ...),
    ((0, 1), 2, ...),
    (0, ..., 1),
    (0, 1, ..., 2),
    (0, ..., 1, 2),
    (0, 1, ..., 2, 3),
    ((0, 1), ..., 2, 3),
    ((0, 1), ..., (2, 3)),
    ]
for test in tests:
    print(test)
    prelim, *postlim = delim_split(test)
    postlim = postlim[0] if postlim else postlim
    print(' ', prelim)
    print(' ', postlim)
    print('\n')